In [1]:
#importando as bibliotecas 
import pandas as pd
import json
import nltk
import time
import re
from bs4 import BeautifulSoup
nltk.download('punkt')
from textblob import TextBlob
import sys
import os.path

[nltk_data] Downloading package punkt to /home/pcorrea/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
inicio = 48761
terminou = 50000

In [3]:
! git clone https://github.com/paulordie/sentimental_ds

fatal: destination path 'sentimental_ds' already exists and is not an empty directory.


In [4]:
%cd sentimental_ds/
%ls

/home/pcorrea/Documents/Ciencia-De-Dados-UEA/programacao-para-ciencia-dedados/scripts/sentimental_ds
dados-curso.csv        main.txt           piloto-2.ipynb
dataset-v2.dat         pc-dataset-v2.csv  prototype_sentimental_analyzer.ipynb
first_code_test.ipynb  pc-dataset-v2.dat  README.md


In [5]:
import json
import pandas as pd
import time
dados = open('dataset-v2.dat', encoding="utf8") #pandas
print("Tipo do dado: ", type(dados))

Tipo do dado:  <class '_io.TextIOWrapper'>


In [6]:
! pip install googletrans

In [7]:
listOfDataBase = list()
listOfData = []
 
def findKeys(dataBase):
    for register in dataBase:
        try:
            cols = register.split('\n')
            listOfDataBase.append(cols[0])
        except UnicodeDecodeError:
            print("Error on uncode decode findkeys")
    print("Find Keys: ",listOfDataBase)

In [8]:
def passToJson():
    i=0
    for i in range(100000): ## tem que colocar a array de todo arquivo
        listOfIssue = listOfDataBase[i]
        i+=1
#         time.sleep(1)
        joinList = json.loads(listOfIssue)
        data = pd.json_normalize(joinList)
        listOfData.append(data)
    return listOfData

In [9]:
def showContent(value, collum):
    i=0
    _data = []
    for i in range(value): ##range que vai trazer todas as colunas
        try:
            res = listOfData[i][collum]
            print(type(res))
            print(listOfData[i][collum]) #averageRating, ratingValue etc
            i = i + 1
            print("line: ", i)
        except KeyError:
            print("Key Error")
            continue

valueFindKeys = findKeys(dados) #valor que vai para def passToJson()
dictionaryOfDate = passToJson()
    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
df = pd.concat(objs=dictionaryOfDate,axis=0)

In [11]:
X = df.iloc[:, 4].values
type(X)
len(X)

100000

In [20]:
X

array(['Comida maravilhosa, lugar do verdadeiro chiclete de camarão. \nO garçom Éric Lima nos atendeu muito bem, pretendemos voltar assim que possível.',
       'Comida boa, drinks legais e um ambiente bem aconchegante. \nProvamos muitas coisas e tudo nos agradou. Principalmente um pato servido em um dos pratos da casa, sensacional!',
       'Excelente pra ir com a família! Ótima Comida, ambiente agradável, bom atendimento e estacionamento. Vale a pena conhecer',
       ...,
       'O Rio Scenarium é uma das baladas mais famosas do Rio de Janeiro. É bem grande e lotada de estrangeiros. A bebida não é tão cara como em outras cidades, por exemplo, São Paulo. O lado negativo é com certeza a fila. Se você não chegar cedo, vai ficar pelo menos 30 minutos esperando para entrar.',
       'Fui por indicação e com boa expectativa!\n116,00 o kilo de salada... Bem caro, né?\nO alface tava murcho, nada de opções de tempero, buffet pequeno...\n12,00 um copo peq de suco. Aff.\nNao valeu, nem pelo am

In [13]:
def clean_tripadvisor(tweet):
    try:
        tweet = BeautifulSoup(tweet, 'lxml').get_text() # o lxml indica o parse pode ser usado para dados de web por ex
    #     tweet = re.sub(r"\b\.\s\\n", '\n', tweet)
    #     tweet = re.sub(r"\b\.\s[A-Za-z0-9]", '\n', tweet)
        tweet = re.sub(r"\b\.\s\\n", '. \n', tweet)
        tweet = re.sub(r"\b\.", '. \n', tweet)    
        return tweet
    except TypeError:
        print("errou no tipo")
        

In [14]:
arquivoToSave = "/home/pcorrea/Documents/Ciencia-De-Dados-UEA/programacao-para-ciencia-dedados/scripts/salvo_poradiradade_vm2-{}-{}.txt".format(inicio, terminou)

def saveDocument(var,resultado,nota,polaridade,fim):
    try:
        with open(arquivoToSave, 'a') as toSave:
            toSave.write('{} - {} subj {} pol {} \n'.format(var,resultado,nota, polaridade))
            if fim == "fim":
                toSave.close()
            print('It was created result: ',arquivoToSave)
                
    except Exception as e:
        print("Error did file ", e)
            

In [15]:
arquivoToSaveError = "/home/pcorrea/Documents/Ciencia-De-Dados-UEA/programacao-para-ciencia-dedados/scripts/erros_poradiradade_vm2-{}-{}.txt".format(inicio, terminou)

def saveDocumentError(var, item,obj):
    try:
        with open(arquivoToSaveError, 'a') as toSave:
            toSave.write('{} - qtd erro {} : {} \n'.format(var,item,obj))
            
            toSave.close()
            print('It was saver error: ',arquivoToSaveError)
                
    except Exception as e:
        print("Error did file ", e)

In [16]:
def transTextBlob(sentences):
    ##textblob
    comentario_blob = TextBlob(sentences)
    print(type(comentario_blob))
    comentario_blob_us = comentario_blob.translate(to='us')
    print("textblob traduziu: ",comentario_blob_us)
    return comentario_blob, comentario_blob_us
    ##textblob

In [17]:
def transGoogle(sentences):
    ##googletrans
    import pandas as pd
    from googletrans import Translator   
    translator = Translator()
    try:
        translations = translator.translate([sentences], dest='en')
        for translation in translations:
            print(type(translation.text))
            print("google traduziu: ", translation.text)
    except AttributeError:
         print("Error googletrans")
    ##googletrans

In [32]:
var = 0
errou = 0
for x in df['reviewBody']:
    ini = time.time()
    var = var + 1
    if var > inicio and var <= terminou:
        print("**************\n")
        print( "var {} x {}".format(var,x))
        print("**************\n")
        #out = clean_tripadvisor("Recife: a beautiful city with a gorgeous culture. Beyond the beauties, we also have a really good food. So, do you really want to eat an awesome steak? If you love steak like I do, you need to know a perfect place. What’s the name of the place? Spettus! You’ll see more than 22 kinds of steaks, all of them prepared by excellent professionals. Steaks from Argentina and Uruguay, and you can choose any kind of steak you want. You’ll find japanese food, vegetables and delicious deserts too! What are you going to find there? A huge place (they fit 700 people), perfect for a good lunch, a dinner with the family and, of course, a happy hour with your friends. Do you like good music? Piano bar. Do you have kids? Playground. Everything you’ll find at Spettus! Do you drink? Be prepared to drink a good wine from France, Italy, Argentina and more! Oh! Do you want to surf on the internet while eating? Do you need an Internet Wi-fi? Spettus is the answer! I went there last year and it was perfect. I love steak and they have a really good variety. And, obvious, it’s delicious! Remember: Spettus!")
        out = clean_tripadvisor(x)
        #################################
        try:
            which = TextBlob(out)
            print("Tipo de which language: ",type(which))
            lang = which.detect_language()
            if lang == 'en':
                print("It is english")
                #################################
                #comentario_blob = TextBlob(out)
    #             print("comentario_blob ",type(which))

                comentario_blob_pt = which.translate(to='pt')
                comentario_blob_us = comentario_blob_pt.translate(to='en')
                print("tipo em english ", type(which))
                print("Original em Inglês: ",which)
                print("Traduzido para Português: ",comentario_blob_pt)
                time.sleep(5)

                for sentence_br, sentence in zip(comentario_blob_pt.sentences, comentario_blob_us.sentences):
                    print("sentence_br ",sentence_br)
                    print(" EN sentence.sentiment.polarity: ",sentence.sentiment.polarity)
                    print("EN sentence.sentiment.subjectivity: ",sentence.sentiment.subjectivity)
                    if sentence_br is None:
                        errou = errou + 1
                        print("errou em {} - {} {}".format(var,errou,x))
                        saveDocumentError(var,errou,x)

                    elif var > inicio and var <= terminou:
                        print('Polaridade\n',sentence_br, ':', sentence.sentiment.polarity)
                        print('subjetividade\n',sentence_br, ':', sentence.sentiment.subjectivity)
                        saveDocument(var,sentence_br,sentence.sentiment.subjectivity,sentence.sentiment.polarity, 'fim')

                    else:
                        print("EN algum erro ocorreu em {}".format(var))
                ################################################

            elif lang == 'pt':
                print("It is portuguese")
                #################################
                comentario_blob = TextBlob(out)
                print("comentario_blob ",type(comentario_blob))

                comentario_blob_us = comentario_blob.translate(to='us')
                print("tipo em english ", type(comentario_blob_us))
                print("Inglês: ",comentario_blob_us)
                time.sleep(5)

                for sentence_br, sentence in zip(comentario_blob.sentences, comentario_blob_us.sentences):
                    print("sentence_br ",sentence_br)
                    print("sentence.sentiment.polarity: ",sentence.sentiment.polarity)
                    print("sentence.sentiment.subjectivity: ",sentence.sentiment.subjectivity)
                    if sentence_br is None:
                        errou = errou + 1
                        print("errou em {} - {} {}".format(var,errou,x))
                        saveDocumentError(var,errou,x)

                    elif var > inicio and var <= terminou:
                        print('Polaridade\n',sentence_br, ':', sentence.sentiment.polarity)
                        print('subjetividade\n',sentence_br, ':', sentence.sentiment.subjectivity)
                        saveDocument(var,sentence_br,sentence.sentiment.subjectivity,sentence.sentiment.polarity, 'fim')

                    else:
                        print("PT algum erro ocorreu em {}".format(var))
                ################################################
            else:
                print("Not english")
        except TypeError:
            print("Errou não é tipo string")
            continue
        
    if var == terminou:
        print("Finalizou em ",var)
        break
    fim = time.time()
    print("tempo final foi ", ini-fim)
   

tempo final foi  -9.5367431640625e-07
tempo final foi  -9.5367431640625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -7.152557373046875e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -7.152557373046875e-07
tempo final foi  -7.152557373046875e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -9.5367431640625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -7.152557373046875e-07
tempo final foi  -9.5367431640625e-07
tempo final foi  -7.152557373046875e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -7.152557373046875e-07
tempo final foi  -7.152557373046875e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -7.152557373046875e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  

tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  0.0
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384

tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  0.0
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384

tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  0.0
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  0.0
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
te

tempo final foi  0.0
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -4.76837158203125e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.384185791015625e-07
tempo final foi  -2.38

...

It is portuguese
comentario_blob  <class 'textblob.blob.TextBlob'>
tipo em english  <class 'textblob.blob.TextBlob'>
Inglês:  Very nice place, cozy atmosphere and delicious food.
 The menu has many options.
 The pizza is also very good.
sentence_br  Lugar muito agradável, ambiente aconchegante e comida deliciosa.
sentence.sentiment.polarity:  0.5266666666666667
sentence.sentiment.subjectivity:  0.9166666666666666
Polaridade
 Lugar muito agradável, ambiente aconchegante e comida deliciosa. : 0.5266666666666667
subjetividade
 Lugar muito agradável, ambiente aconchegante e comida deliciosa. : 0.9166666666666666
It was created result:  /home/pcorrea/Documents/Ciencia-De-Dados-UEA/programacao-para-ciencia-dedados/scripts/salvo_poradiradade_vm2-40715-50000.txt
sentence_br  O cardápio possui muitas opções.
sentence.sentiment.polarity:  0.5
sentence.sentiment.subjectivity:  0.5
Polaridade
 O cardápio possui muitas opções. : 0.5
subjetividade
 O cardápio possui muitas opções. : 0.5
It was creat

It is portuguese
comentario_blob  <class 'textblob.blob.TextBlob'>
tipo em english  <class 'textblob.blob.TextBlob'>
Inglês:  What won me over was the flexibility of the manager AND chef for allowing the client to choose the best side dishes.
sentence_br  O que me conquistou foi a flexibilidade do gerente E chefe de cozinha por permitir que o cliente escolhesse os melhores acompanhamentos.
sentence.sentiment.polarity:  1.0
sentence.sentiment.subjectivity:  0.3
Polaridade
 O que me conquistou foi a flexibilidade do gerente E chefe de cozinha por permitir que o cliente escolhesse os melhores acompanhamentos. : 1.0
subjetividade
 O que me conquistou foi a flexibilidade do gerente E chefe de cozinha por permitir que o cliente escolhesse os melhores acompanhamentos. : 0.3
It was created result:  /home/pcorrea/Documents/Ciencia-De-Dados-UEA/programacao-para-ciencia-dedados/scripts/salvo_poradiradade_vm2-40715-50000.txt
tempo final foi  -5.834748029708862
**************

var 48797 x Comida go

It is portuguese
comentario_blob  <class 'textblob.blob.TextBlob'>
tipo em english  <class 'textblob.blob.TextBlob'>
Inglês:  I went at lunchtime during the week, at the executive buffet.
 The food was very good, with a good variety of salads and hot dishes, in addition to the pasta island.
 The desserts were also great, very varied, with options such as creamy coconut, ambrosia, milk pudding, passion fruit pie and chocolate mousse.
 The service was very fast, and the orange juice was also very good.
 A great cost x benefit.
 I recommend!
sentence_br  Fui na hora do almoço durante a semana, no buffet executivo.
sentence.sentiment.polarity:  0.0
sentence.sentiment.subjectivity:  0.0
Polaridade
 Fui na hora do almoço durante a semana, no buffet executivo. : 0.0
subjetividade
 Fui na hora do almoço durante a semana, no buffet executivo. : 0.0
It was created result:  /home/pcorrea/Documents/Ciencia-De-Dados-UEA/programacao-para-ciencia-dedados/scripts/salvo_poradiradade_vm2-40715-50000.txt

sentence_br  Muito, muito bom!
sentence.sentiment.polarity:  1.0
sentence.sentiment.subjectivity:  0.7800000000000001
Polaridade
 Muito, muito bom! : 1.0
subjetividade
 Muito, muito bom! : 0.7800000000000001
It was created result:  /home/pcorrea/Documents/Ciencia-De-Dados-UEA/programacao-para-ciencia-dedados/scripts/salvo_poradiradade_vm2-40715-50000.txt
sentence_br  Melhor japonês dessa cidade sem sombra de dúvida!
sentence.sentiment.polarity:  0.5
sentence.sentiment.subjectivity:  0.15
Polaridade
 Melhor japonês dessa cidade sem sombra de dúvida! : 0.5
subjetividade
 Melhor japonês dessa cidade sem sombra de dúvida! : 0.15
It was created result:  /home/pcorrea/Documents/Ciencia-De-Dados-UEA/programacao-para-ciencia-dedados/scripts/salvo_poradiradade_vm2-40715-50000.txt
sentence_br  O ambiente é muito bonito e confortável, a comida é divina!
sentence.sentiment.polarity:  0.675
sentence.sentiment.subjectivity:  0.9
Polaridade
 O ambiente é muito bonito e confortável, a comida é divina!

HTTPError: HTTP Error 429: Too Many Requests